## 공정 4개 나눠서 모델링

전처리: 0814_preprocessing_sep.ipynb

In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt 
%matplotlib inline
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# csv 불러오기 (0814_preprocessing_sep.ipynb에서 전처리된 데이터)
train_data = pd.read_csv('sep_model_train_data.csv')
test_data = pd.read_csv('sep_model_test_data.csv')

In [3]:
### 총시간 대비 비율 변수
def calculate_total_time_and_ratios(data):
    data['total_time'] = (
        data['Machine Tact time Collect Result_Dam'] +
        data['Machine Tact time Collect Result_Fill1'] +
        data['Machine Tact time Collect Result_Fill2'] +
        data['Chamber Temp. Unit Time_AutoClave']
    )
    data['time_ratio_Dam'] = (data['Machine Tact time Collect Result_Dam'] / data['total_time']).round(3)
    data['time_ratio_Fill1'] = (data['Machine Tact time Collect Result_Fill1'] / data['total_time']).round(3)
    data['time_ratio_Fill2'] = (data['Machine Tact time Collect Result_Fill2'] / data['total_time']).round(3)
    data['time_ratio_AutoClave'] = (data['Chamber Temp. Unit Time_AutoClave'] / data['total_time']).round(3)
    return data

# train_data와 test_data에 함수 적용
train_data = calculate_total_time_and_ratios(train_data)
test_data = calculate_total_time_and_ratios(test_data)

In [4]:
# 변수 제거
train_data.drop(columns=[
    'total_time'
    , 'Machine Tact time Collect Result_Dam'
    , 'Machine Tact time Collect Result_Fill1'
    , 'Machine Tact time Collect Result_Fill2'
    , 'Chamber Temp. Unit Time_AutoClave'], inplace=True)

test_data.drop(columns=[
    'total_time'
    , 'Machine Tact time Collect Result_Dam'
    , 'Machine Tact time Collect Result_Fill1'
    , 'Machine Tact time Collect Result_Fill2'
    , 'Chamber Temp. Unit Time_AutoClave'], inplace=True)

### 공통 변수

In [5]:
variables = [
    "Equipment_same_num",
    "PalletID",
    "Production_Qty",
    "model_receip_encoded",
    "workorder_receip_encoded"
]

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [6]:
# 상관계수 행렬 계산
correlation_matrix = filtered_data.corr()

# 자기자신을 제외하고 특정 값 이상인 조합 찾기
strong_correlations = correlation_matrix[(correlation_matrix >= 0.8) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
strong_correlations_pairs = strong_correlations_pairs[strong_correlations_pairs['Correlation'] >= 0.7]
print(strong_correlations_pairs)

       Variable 1      Variable 2  Correlation
0        PalletID  Production_Qty     0.944591
1  Production_Qty        PalletID     0.944591


In [7]:
# 열 삭제
train_data.drop("PalletID", axis=1, inplace=True)
test_data.drop("PalletID", axis=1, inplace=True)

-> 상관관계가 높은 이유: Production_Qty가 0이고 PalletID가 1인 경우는 없음 (Production_Qty가 1인 경우 안에 PalletID가 1인 경우가 모두 포함됨). 따라서 PalletID 컬럼을 삭제.

In [8]:
# 공통 변수 리스트
same_variables_train = [
    "target",
    "Equipment_same_num",
    "Production_Qty",
    "model_receip_encoded",
    "workorder_receip_encoded"
]

same_variables_test = [
    "Set ID",
    "target",
    "Equipment_same_num",
    "Production_Qty",
    "model_receip_encoded",
    "workorder_receip_encoded"
]

### dam

In [9]:
# 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Dam').columns

# 필터링된 열 이름 출력
print("<Dam 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Dam 공정 관련 변수>
CURE SPEED Collect Result_Dam
DISCHARGED SPEED OF RESIN Collect Result_Dam
Head Clean Position Z Collect Result_Dam
Head Purge Position Z Collect Result_Dam
Head Zero Position Y Collect Result_Dam
head_normal_vector_stage1_Dam
head_normal_vector_stage2_Dam
head_normal_vector_stage3_Dam
cure_end_position_XZΘ_Dam
cure_start_position_XΘ_Dam
total_circle_distance_speed_Dam
total_line_distance_speed_Dam
volume_time_multip_avg_Dam
average_thickness_Dam
time_ratio_Dam


In [10]:
variables = [
    "CURE SPEED Collect Result_Dam",
    "DISCHARGED SPEED OF RESIN Collect Result_Dam",
    "Head Clean Position Z Collect Result_Dam",
    "Head Purge Position Z Collect Result_Dam",
    "Head Zero Position Y Collect Result_Dam",
    #"head_normal_vector_stage1_Dam",
    #"head_normal_vector_stage2_Dam",
    #"head_normal_vector_stage3_Dam",
    "cure_end_position_XZΘ_Dam",
    "cure_start_position_XΘ_Dam",
    "total_circle_distance_speed_Dam",
    "total_line_distance_speed_Dam",
    "volume_time_multip_avg_Dam",
    "average_thickness_Dam"
]

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [11]:
# 상관계수 행렬 계산
correlation_matrix = filtered_data.corr()

# 자기자신을 제외하고 특정 값 이상인 조합 찾기
strong_correlations = correlation_matrix[(correlation_matrix >= 0.8) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
strong_correlations_pairs = strong_correlations_pairs[strong_correlations_pairs['Correlation'] >= 0.7]
print(strong_correlations_pairs)

Empty DataFrame
Columns: [Variable 1, Variable 2, Correlation]
Index: []


In [12]:
# 드랍할 열 목록
columns_to_drop = [
    'head_normal_vector_stage1_Dam',
    'head_normal_vector_stage2_Dam',
    'head_normal_vector_stage3_Dam',
]

# 열 삭제
train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [13]:
### dam 데이터셋
# 열 이름 필터링 후 공통 변수와 결합
Process_Desc_col = train_data.filter(like='_Dam').columns

# train
final_columns_train = list(Process_Desc_col) + same_variables_train
df_train_dam = train_data[final_columns_train]

# test 
final_columns_test = list(Process_Desc_col) + same_variables_test
df_test_dam = test_data[final_columns_test]

In [14]:
df_train_dam.columns

Index(['CURE SPEED Collect Result_Dam',
       'DISCHARGED SPEED OF RESIN Collect Result_Dam',
       'Head Clean Position Z Collect Result_Dam',
       'Head Purge Position Z Collect Result_Dam',
       'Head Zero Position Y Collect Result_Dam', 'cure_end_position_XZΘ_Dam',
       'cure_start_position_XΘ_Dam', 'total_circle_distance_speed_Dam',
       'total_line_distance_speed_Dam', 'volume_time_multip_avg_Dam',
       'average_thickness_Dam', 'time_ratio_Dam', 'target',
       'Equipment_same_num', 'Production_Qty', 'model_receip_encoded',
       'workorder_receip_encoded'],
      dtype='object')

In [15]:
df_test_dam.columns

Index(['CURE SPEED Collect Result_Dam',
       'DISCHARGED SPEED OF RESIN Collect Result_Dam',
       'Head Clean Position Z Collect Result_Dam',
       'Head Purge Position Z Collect Result_Dam',
       'Head Zero Position Y Collect Result_Dam', 'cure_end_position_XZΘ_Dam',
       'cure_start_position_XΘ_Dam', 'total_circle_distance_speed_Dam',
       'total_line_distance_speed_Dam', 'volume_time_multip_avg_Dam',
       'average_thickness_Dam', 'time_ratio_Dam', 'Set ID', 'target',
       'Equipment_same_num', 'Production_Qty', 'model_receip_encoded',
       'workorder_receip_encoded'],
      dtype='object')

### fill1

In [16]:
# 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Fill1').columns

# 필터링된 열 이름 출력
print("<Fill1 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Fill1 공정 관련 변수>
DISCHARGED SPEED OF RESIN Collect Result_Fill1
Head Purge Position Z Collect Result_Fill1
head_normal_vector_stage1_Fill1
head_normal_vector_stage2_Fill1
head_normal_vector_stage3_Fill1
volume_time_multip_avg_Fill1
time_ratio_Fill1


In [17]:
variables = [
    "DISCHARGED SPEED OF RESIN Collect Result_Fill1",
    "Head Purge Position Z Collect Result_Fill1",
    "head_normal_vector_stage1_Fill1",
    #"head_normal_vector_stage2_Fill1",
    #"head_normal_vector_stage3_Fill1",
    "volume_time_multip_avg_Fill1",
    "time_ratio_Fill1"
]

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [18]:
# 상관계수 행렬 계산
correlation_matrix = filtered_data.corr()

# 자기자신을 제외하고 특정 값 이상인 조합 찾기
strong_correlations = correlation_matrix[(correlation_matrix >= 0.8) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
strong_correlations_pairs = strong_correlations_pairs[strong_correlations_pairs['Correlation'] >= 0.7]
print(strong_correlations_pairs)

Empty DataFrame
Columns: [Variable 1, Variable 2, Correlation]
Index: []


In [19]:
# 드랍할 열 목록
columns_to_drop = [
    "head_normal_vector_stage2_Fill1",
    "head_normal_vector_stage3_Fill1"
]

# 열 삭제
train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [20]:
### fill1 데이터셋
# 열 이름 필터링 후 공통 변수와 결합
Process_Desc_col = train_data.filter(like='_Fill1').columns

# train
final_columns_train = list(Process_Desc_col) + same_variables_train
df_train_fill1 = train_data[final_columns_train]

# test 
final_columns_test = list(Process_Desc_col) + same_variables_test
df_test_fill1 = test_data[final_columns_test]

In [21]:
df_train_fill1.columns

Index(['DISCHARGED SPEED OF RESIN Collect Result_Fill1',
       'Head Purge Position Z Collect Result_Fill1',
       'head_normal_vector_stage1_Fill1', 'volume_time_multip_avg_Fill1',
       'time_ratio_Fill1', 'target', 'Equipment_same_num', 'Production_Qty',
       'model_receip_encoded', 'workorder_receip_encoded'],
      dtype='object')

In [22]:
df_test_fill1.columns

Index(['DISCHARGED SPEED OF RESIN Collect Result_Fill1',
       'Head Purge Position Z Collect Result_Fill1',
       'head_normal_vector_stage1_Fill1', 'volume_time_multip_avg_Fill1',
       'time_ratio_Fill1', 'Set ID', 'target', 'Equipment_same_num',
       'Production_Qty', 'model_receip_encoded', 'workorder_receip_encoded'],
      dtype='object')

### fill2

In [23]:
# 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Fill2').columns

# 필터링된 열 이름 출력
print("<Fill2 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Fill2 공정 관련 변수>
CURE SPEED Collect Result_Fill2
CURE STANDBY POSITION Z Collect Result_Fill2
Head Purge Position Z Collect Result_Fill2
head_normal_vector_stage1_Fill2
head_normal_vector_stage2_Fill2
head_normal_vector_stage3_Fill2
cure_end_position_XZ_Fill2
cure_start_position_XZ_Fill2
time_ratio_Fill2


In [24]:
variables = [
    "CURE SPEED Collect Result_Fill2",
    "CURE STANDBY POSITION Z Collect Result_Fill2",
    "Head Purge Position Z Collect Result_Fill2",
    "head_normal_vector_stage1_Fill2",
    #"head_normal_vector_stage2_Fill2",
    #"head_normal_vector_stage3_Fill2",
    "cure_end_position_XZ_Fill2",
    "cure_start_position_XZ_Fill2",
    "time_ratio_Fill2"
]

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [25]:
# 상관계수 행렬 계산
correlation_matrix = filtered_data.corr()

# 자기자신을 제외하고 특정 값 이상인 조합 찾기
strong_correlations = correlation_matrix[(correlation_matrix >= 0.8) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
strong_correlations_pairs = strong_correlations_pairs[strong_correlations_pairs['Correlation'] >= 0.7]
print(strong_correlations_pairs)

                                     Variable 1  \
0  CURE STANDBY POSITION Z Collect Result_Fill2   
1                  cure_start_position_XZ_Fill2   

                                     Variable 2  Correlation  
0                  cure_start_position_XZ_Fill2     0.846498  
1  CURE STANDBY POSITION Z Collect Result_Fill2     0.846498  


In [26]:
# 드랍할 열 목록
columns_to_drop = [
    "head_normal_vector_stage2_Fill2",
    "head_normal_vector_stage3_Fill2",
]

# 열 삭제
train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [27]:
### fill2 데이터셋
# 열 이름 필터링 후 공통 변수와 결합
Process_Desc_col = train_data.filter(like='_Fill2').columns

# train
final_columns_train = list(Process_Desc_col) + same_variables_train
df_train_fill2 = train_data[final_columns_train]

# test 
final_columns_test = list(Process_Desc_col) + same_variables_test
df_test_fill2 = test_data[final_columns_test]

In [28]:
df_train_fill2.columns

Index(['CURE SPEED Collect Result_Fill2',
       'CURE STANDBY POSITION Z Collect Result_Fill2',
       'Head Purge Position Z Collect Result_Fill2',
       'head_normal_vector_stage1_Fill2', 'cure_end_position_XZ_Fill2',
       'cure_start_position_XZ_Fill2', 'time_ratio_Fill2', 'target',
       'Equipment_same_num', 'Production_Qty', 'model_receip_encoded',
       'workorder_receip_encoded'],
      dtype='object')

In [29]:
df_test_fill2.columns

Index(['CURE SPEED Collect Result_Fill2',
       'CURE STANDBY POSITION Z Collect Result_Fill2',
       'Head Purge Position Z Collect Result_Fill2',
       'head_normal_vector_stage1_Fill2', 'cure_end_position_XZ_Fill2',
       'cure_start_position_XZ_Fill2', 'time_ratio_Fill2', 'Set ID', 'target',
       'Equipment_same_num', 'Production_Qty', 'model_receip_encoded',
       'workorder_receip_encoded'],
      dtype='object')

### autoclave

In [30]:
# 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("<AutoClave 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<AutoClave 공정 관련 변수>
Chamber Temp. Collect Result_AutoClave
avg_pressure_time_AutoClave
time_ratio_AutoClave


In [31]:
variables = [
    "Chamber Temp. Collect Result_AutoClave",
    "avg_pressure_time_AutoClave",
    "time_ratio_AutoClave",
]

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [32]:
# 상관계수 행렬 계산
correlation_matrix = filtered_data.corr()

# 자기자신을 제외하고 특정 값 이상인 조합 찾기
strong_correlations = correlation_matrix[(correlation_matrix >= 0.8) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
strong_correlations_pairs = strong_correlations_pairs[strong_correlations_pairs['Correlation'] >= 0.7]
print(strong_correlations_pairs)

Empty DataFrame
Columns: [Variable 1, Variable 2, Correlation]
Index: []


In [33]:
### autoclave 데이터셋
# 열 이름 필터링 후 공통 변수와 결합
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# train
final_columns_train = list(Process_Desc_col) + same_variables_train
df_train_autoclave = train_data[final_columns_train]

# test 
final_columns_test = list(Process_Desc_col) + same_variables_test
df_test_autoclave = test_data[final_columns_test]

In [34]:
df_train_autoclave.columns

Index(['Chamber Temp. Collect Result_AutoClave', 'avg_pressure_time_AutoClave',
       'time_ratio_AutoClave', 'target', 'Equipment_same_num',
       'Production_Qty', 'model_receip_encoded', 'workorder_receip_encoded'],
      dtype='object')

In [35]:
df_test_autoclave.columns

Index(['Chamber Temp. Collect Result_AutoClave', 'avg_pressure_time_AutoClave',
       'time_ratio_AutoClave', 'Set ID', 'target', 'Equipment_same_num',
       'Production_Qty', 'model_receip_encoded', 'workorder_receip_encoded'],
      dtype='object')

### modeling

In [36]:
### 모델 정의
class_weights = {'Normal': 1, 'AbNormal': 5}  # 클래스의 비율에 따라 가중치 설정

model_Dam = RandomForestClassifier(n_estimators=900, class_weight=class_weights, random_state=42)
model_AutoClave = RandomForestClassifier(n_estimators=900, class_weight=class_weights, random_state=42)
model_Fill1 = RandomForestClassifier(n_estimators=900, class_weight=class_weights, random_state=42)
model_Fill2 = RandomForestClassifier(n_estimators=900, class_weight=class_weights, random_state=42)

In [37]:
# 제출 데이터 읽어오기
df_sub = pd.read_csv("submission.csv")

##### dam

In [38]:
### 모델 학습
# 타깃값 분리
y_train = df_train_dam['target']
x_train = df_train_dam.drop(columns='target')

# 랜덤 포레스트 학습
model_Dam.fit(x_train, y_train)

RandomForestClassifier(class_weight={'AbNormal': 5, 'Normal': 1},
                       n_estimators=900, random_state=42)

In [39]:
### 모델 예측
# 'Set ID'와 'target' 컬럼 분리
set_id = df_test_dam['Set ID']  
target = df_test_dam['target']   

x_test = df_test_dam.drop(columns=['Set ID', 'target'])

test_pred = model_Dam.predict(x_test)
test_pred

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

In [40]:
### 예측 컬럼 생성
df_sub["target_dam"] = test_pred

In [41]:
df_sub["target_dam"].value_counts()

target_dam
Normal      16973
AbNormal      388
Name: count, dtype: int64

##### fill1

In [42]:
### 모델 학습
# 타깃값 분리
y_train = df_train_fill1['target']
x_train = df_train_fill1.drop(columns='target')

# 랜덤 포레스트 학습
model_Fill1.fit(x_train, y_train)

RandomForestClassifier(class_weight={'AbNormal': 5, 'Normal': 1},
                       n_estimators=900, random_state=42)

In [43]:
### 모델 예측
# 'Set ID'와 'target' 컬럼 분리
set_id = df_test_fill1['Set ID']  
target = df_test_fill1['target']   

x_test = df_test_fill1.drop(columns=['Set ID', 'target'])

test_pred = model_Fill1.predict(x_test)
test_pred

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

In [44]:
### 예측 컬럼 생성
df_sub["target_fill1"] = test_pred

In [45]:
df_sub["target_fill1"].value_counts()

target_fill1
Normal      16863
AbNormal      498
Name: count, dtype: int64

##### fill2

In [46]:
### 모델 학습
# 타깃값 분리
y_train = df_train_fill2['target']
x_train = df_train_fill2.drop(columns='target')

# 랜덤 포레스트 학습
model_Fill2.fit(x_train, y_train)

RandomForestClassifier(class_weight={'AbNormal': 5, 'Normal': 1},
                       n_estimators=900, random_state=42)

In [47]:
### 모델 예측
# 'Set ID'와 'target' 컬럼 분리
set_id = df_test_fill2['Set ID']  
target = df_test_fill2['target']   

x_test = df_test_fill2.drop(columns=['Set ID', 'target'])

test_pred = model_Fill2.predict(x_test)
test_pred

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

In [48]:
### 예측 컬럼 생성
df_sub["target_fill2"] = test_pred

In [49]:
df_sub["target_fill2"].value_counts()

target_fill2
Normal      17155
AbNormal      206
Name: count, dtype: int64

##### autoclave

In [50]:
### 모델 학습
# 타깃값 분리
y_train = df_train_autoclave['target']
x_train = df_train_autoclave.drop(columns='target')

# 랜덤 포레스트 학습
model_AutoClave.fit(x_train, y_train)

RandomForestClassifier(class_weight={'AbNormal': 5, 'Normal': 1},
                       n_estimators=900, random_state=42)

In [51]:
### 모델 예측
# 'Set ID'와 'target' 컬럼 분리
set_id = df_test_autoclave['Set ID']  
target = df_test_autoclave['target']   

x_test = df_test_autoclave.drop(columns=['Set ID', 'target'])

test_pred = model_AutoClave.predict(x_test)
test_pred

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

In [52]:
### 예측 컬럼 생성
df_sub["target_autoclave"] = test_pred

In [53]:
df_sub["target_autoclave"].value_counts()

target_autoclave
Normal      16362
AbNormal      999
Name: count, dtype: int64

##### result

In [56]:
### 1개 이상의 공정에서 abnormal이면 abnormal 입력
df_sub['target'] = df_sub[['target_dam', 'target_fill1', 'target_fill2', 'target_autoclave']].apply(lambda row: 'AbNormal' 
                                                                                                    if (row == 'AbNormal').sum() >= 1 
                                                                                                    else 'Normal', axis=1)

In [57]:
df_sub['target'].value_counts()

target
Normal      15688
AbNormal     1673
Name: count, dtype: int64

In [58]:
# 삭제할 열 목록
columns_to_drop = ['target_dam', 'target_fill1', 'target_fill2', 'target_autoclave']

# 열 삭제
df_sub.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df_sub.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [59]:
# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [60]:
# 제출 파일 저장
df_sub.to_csv("0814_rf_sep_submission2.csv", index=False)

In [61]:
### 2개 이상의 공정에서 abnormal이면 abnormal 입력
'''
send files ['code.ipynb', 'submission.csv'] for grade...
waiting result...
done!

Score: None
Duration: 1.858 seconds
=== Message ===
작성하신 답안 제출이 완료되었습니다.
Public Score : 0.1708860759493671
'''

"\nsend files ['code.ipynb', 'submission.csv'] for grade...\nwaiting result...\ndone!\n\nScore: None\nDuration: 1.858 seconds\n=== Message ===\n작성하신 답안 제출이 완료되었습니다.\nPublic Score : 0.1708860759493671\n"

In [62]:
### 1개 이상의 공정에서 abnormal이면 abnormal 입력
'''
send files ['code.ipynb', 'submission.csv'] for grade...
waiting result...
done!

Score: None
Duration: 2.077 seconds
=== Message ===
작성하신 답안 제출이 완료되었습니다.
Public Score : 0.19726858877086492
'''

"\nsend files ['code.ipynb', 'submission.csv'] for grade...\nwaiting result...\ndone!\n\nScore: None\nDuration: 2.077 seconds\n=== Message ===\n작성하신 답안 제출이 완료되었습니다.\nPublic Score : 0.19726858877086492\n"